In [8]:
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import DAGMetric
from deepeval.metrics.dag import (
    BinaryJudgementNode,
    DeepAcyclicGraph,
    NonBinaryJudgementNode,
    TaskNode,
    VerdictNode,
)
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

correct_order_node = NonBinaryJudgementNode(
    criteria="Are the summary headings in the correct order: 'intro' => 'body' => 'conclusion'?",
    children=[
        VerdictNode(verdict="Yes", score=10),
        VerdictNode(verdict="Two are out of order", score=4),
        VerdictNode(verdict="All out of order", score=2),
    ],
)

correct_headings_node = BinaryJudgementNode(
    criteria="Does the summary headings contain all three: 'intro', 'body', and 'conclusion'?",
    children=[
        VerdictNode(verdict=False, score=0),
        VerdictNode(verdict=True, child=correct_order_node),
    ],
)

extract_headings_node = TaskNode(
    instructions="Extract all headings in `actual_output`",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    output_label="Summary headings",
    children=[correct_headings_node, correct_order_node],
)

# Initialize the DAG
dag = DeepAcyclicGraph(root_nodes=[extract_headings_node])

# Create metric!
metric = DAGMetric(name="Summarization", dag=dag)

In [4]:
from ollama import ChatResponse, chat


def query(prompt: str) -> str:
    response: ChatResponse = chat(
        model="llama3.2",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return response.message.content

In [30]:
prompt = "provide a summary about cats. keep it in a paragraph, provide intro, body and conclusion"
response = query(prompt)
print(response)

**Introduction**

Cats are one of the most popular pets worldwide, known for their independence, agility, and affectionate nature. With over 70 recognized breeds, cats have been human companions for thousands of years, providing entertainment, comfort, and loyalty to their owners. From ancient Egyptian worshippers to modern-day cat enthusiasts, felines have captured the hearts of people across cultures and generations.

**Body**

Cats are characterized by their unique physical features, including their slender bodies, short legs, and retractable claws. Their soft, fur coats come in a wide range of colors and patterns, making each individual cat distinct. Domesticated cats are known for their playful, curious nature, often exhibiting behaviors such as pouncing on toys, scratching furniture, and kneading with their paws. Cats also possess exceptional hearing and vision abilities, allowing them to navigate and hunt with ease in the dark. Despite their independent reputation, many cats for

In [31]:
dataset = EvaluationDataset(
    test_cases=[LLMTestCase(input=prompt, actual_output=response)]
)

In [32]:
evaluate(dataset, [metric])

✨ You're running DeepEval's latest Summarization (DAG) Metric! (using deepseek-r1:8b (Ollama), strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |████████████████████████████████████████████████|100% (1/1) [Time Taken: 00:10, 10.50s/test case]



Metrics Summary

  - ✅ Summarization (DAG) (score: 1.0, threshold: 0.5, strict: False, evaluation model: deepseek-r1:8b (Ollama), reason: The score is 1.0 because all summary headings are correctly structured and follow the required order., error: None)

For test case:

  - input: provide a summary about cats. keep it in a paragraph, provide intro, body and conclusion
  - actual output: **Introduction**

Cats are one of the most popular pets worldwide, known for their independence, agility, and affectionate nature. With over 70 recognized breeds, cats have been human companions for thousands of years, providing entertainment, comfort, and loyalty to their owners. From ancient Egyptian worshippers to modern-day cat enthusiasts, felines have captured the hearts of people across cultures and generations.

**Body**

Cats are characterized by their unique physical features, including their slender bodies, short legs, and retractable claws. Their soft, fur coats come in a wide range of col

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Summarization (DAG)', threshold=0.5, success=True, score=1.0, reason='The score is 1.0 because all summary headings are correctly structured and follow the required order.', strict_mode=False, evaluation_model='deepseek-r1:8b (Ollama)', error=None, evaluation_cost=0.0, verbose_logs="______________________\n| TaskNode | Level == 0 |\n*******************************\nLabel: None\n\nInstructions:\nExtract all headings in `actual_output`\n\nSummary headings:\nCats are one of the most popular pets worldwide\n \n \n__________________________________\n| BinaryJudgementNode | Level == 1 |\n************************************************\nLabel: None\n\nCriteria:\nDoes the summary headings contain all three: 'intro', 'body', and 'conclusion'?\n\nVerdict: True\nReason: The structure contains all three sections: introduction, body, and conclusion.\n \n \n____________________________________

In [28]:
prompt = "summarize the facts about cats"
response = """Cats are popular domesticated animals known for their agility, playfulness, and affectionate nature. They have a unique physical appearance, with characteristics such as whiskers, pointy ears, and retractable claws.
Cats are also highly adaptable and can thrive in various environments, from apartments to large homes with yards. Their self-cleaning habits and low-maintenance lifestyle make them an attractive pet for many people. In conclusion, cats are fascinating animals that have become a beloved companion for millions of people around the world. Their unique appearance, playful personalities, and adaptability make them a popular choice as pets.
"""
print(response)

Cats are popular domesticated animals known for their agility, playfulness, and affectionate nature. They have a unique physical appearance, with characteristics such as whiskers, pointy ears, and retractable claws.
Cats are also highly adaptable and can thrive in various environments, from apartments to large homes with yards. Their self-cleaning habits and low-maintenance lifestyle make them an attractive pet for many people. In conclusion, cats are fascinating animals that have become a beloved companion for millions of people around the world. Their unique appearance, playful personalities, and adaptability make them a popular choice as pets.



In [29]:
dataset = EvaluationDataset(
    test_cases=[LLMTestCase(input=prompt, actual_output=response)]
)
evaluate(dataset, [metric])

✨ You're running DeepEval's latest Summarization (DAG) Metric! (using deepseek-r1:8b (Ollama), strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |████████████████████████████████████████████████|100% (1/1) [Time Taken: 00:10, 10.46s/test case]



Metrics Summary

  - ✅ Summarization (DAG) (score: 1.0, threshold: 0.5, strict: False, evaluation model: deepseek-r1:8b (Ollama), reason: The score is 1.0 because all summary headings are present and correctly ordered as 'intro', 'body', and 'conclusion'., error: None)

For test case:

  - input: summarize the facts about cats
  - actual output: Cats are popular domesticated animals known for their agility, playfulness, and affectionate nature. They have a unique physical appearance, with characteristics such as whiskers, pointy ears, and retractable claws.
Cats are also highly adaptable and can thrive in various environments, from apartments to large homes with yards. Their self-cleaning habits and low-maintenance lifestyle make them an attractive pet for many people. In conclusion, cats are fascinating animals that have become a beloved companion for millions of people around the world. Their unique appearance, playful personalities, and adaptability make them a popular choice as p

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Summarization (DAG)', threshold=0.5, success=True, score=1.0, reason="The score is 1.0 because all summary headings are present and correctly ordered as 'intro', 'body', and 'conclusion'.", strict_mode=False, evaluation_model='deepseek-r1:8b (Ollama)', error=None, evaluation_cost=0.0, verbose_logs="______________________\n| TaskNode | Level == 0 |\n*******************************\nLabel: None\n\nInstructions:\nExtract all headings in `actual_output`\n\nSummary headings:\nCats are popular domesticated animals known for their agility, playfulness, and affectionate nature. They have a unique physical appearance, with characteristics such as whiskers, pointy ears, and retractable claws.\n \n \n__________________________________\n| BinaryJudgementNode | Level == 1 |\n************************************************\nLabel: None\n\nCriteria:\nDoes the summary headings contain all three: